In [ ]:
import os
import langchain
from langchain import PromptTemplate, LLMChain

# huggingface

In [ ]:
from langchain import HuggingFacePipeline

llm = HuggingFacePipeline.from_model_id(
    model_id="bigscience/bloom-560m",
    task="text-generation",
    model_kwargs={"temperature": 0, "max_length": 64},
)


# Integrate the model in an LLMChain
from langchain import PromptTemplate, LLMChain

template = """Question: {question}

Answer: Let's think step by step."""
prompt = PromptTemplate(template=template, input_variables=["question"])

llm_chain = LLMChain(prompt=prompt, llm=llm)

question = "What is electroencephalography?"

print(llm_chain.run(question))

# galactica

In [ ]:
import galai as gal

model = gal.load_model("standard")
# model.generate("Scaled dot product attention:\n\n\\[")
# Scaled dot product attention:\n\n\\[ \\displaystyle\\text{Attention}(Q,K,V)=\\text{softmax}(\\frac{QK^{T}}{\\sqrt{d_{k}}}%\n)V \\]

In [ ]:
model.generate("from this list, [vodka, strawberries, corn, peas],create a new python list that ONLY includes produce <work>")

In [ ]:
TEXT = "vodka, strawberries, corn, peas, cherries, sodapop"
model.generate( '\n\nQuestion: Of the items in this list, \n\n vodka, strawberries, corn, peas, cherries, diet coke, \n\n which can grow in a garden?\n\nAnswer:')


In [ ]:
model.generate("a plant compatability matrix is a a python matrix and will have a score of -1 for negative relationship between plants, 0 for neutral relationship between plants, and 1 for a positive relationship between plants. create a python array of plant compatibility between the plants listed: " + str(plant_list))

# openai + langchain

In [ ]:
# plant compatiblity context source: https://waldenlabs.com/the-ultimate-companion-planting-guide-chart/

In [ ]:
import os
from langchain.chat_models import ChatOpenAI

file_path = 'C:/Users/dheym/OneDrive/Documents/api_keys/openai_api_keys.txt'
with open(file_path, 'r') as file:
    OPENAI_API_KEY = file.read()

os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY
#If you'd prefer not to set an environment variable you can pass the key in directly via the openai_api_key named parameter when initiating the OpenAI LLM class:


chat = ChatOpenAI()



In [ ]:
def parse_and_evaluate_text(text):
    # Find the indices of the opening and closing brackets
    opening_bracket_index = text.find("[")
    closing_bracket_index = text.find("]")

    if opening_bracket_index != -1 and closing_bracket_index != -1:
        # Extract the text within the brackets
        extracted_list = "[" + text[opening_bracket_index + 1: closing_bracket_index] + "]"
        # Return the evaluated text list
        return eval(extracted_list)
        

    else:
        print("Error with parsing plant list")
        return None

In [ ]:
from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    AIMessagePromptTemplate,
    HumanMessagePromptTemplate,
)


def parse_and_evaluate_text(text):
    # Find the indices of the opening and closing brackets
    opening_bracket_index = text.find("[")
    closing_bracket_index = text.find("]")

    if opening_bracket_index != -1 and closing_bracket_index != -1:
        # Extract the text within the brackets
        extracted_list = "[" + text[opening_bracket_index + 1: closing_bracket_index] + "]"
        # Return the evaluated text list
        return eval(extracted_list)
        

    else:
        print("Error with parsing plant list")
        return None
    
def chat_response(template, prompt_text):
    system_message_prompt = SystemMessagePromptTemplate.from_template(template)
    human_template="{text}"
    human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)
    chat_prompt = ChatPromptTemplate.from_messages([system_message_prompt, human_message_prompt])
    response = chat(chat_prompt.format_prompt(text= prompt_text).to_messages())
    return response

# get the plant list from user input
def get_plant_list(input_plant_text):
    template="You are a helpful assistant that knows all about gardening and plants and python data structures."
    text = 'which of the elements of this list can be grown in a garden, [' + input_plant_text + ']? Return JUST a python list object containing the elements that can be grown in a garden. Do not include any other text or explanation.'
    plant_list_text = chat_response(template, text)
    plant_list = parse_and_evaluate_text(plant_list_text.content)
    print(plant_list)
    return plant_list

# get compatability matrix for companion planting
def get_compatibility_matrix(plant_list):
    # Convert the compatibility matrix to a string
    with open('compatibilities_text.txt', 'r') as file:
        # Read the contents of the file
        compatibility_text = file.read()
    plant_comp_context = compatibility_text
    template="You are a helpful assistant that knows all about gardening, companion planting, and python data structures- specifically compatibility matrices."
    text = 'from this list of plants, [' + str(plant_list) + '], Return JUST a python array (with values separated by commas like this: [[0,1],[1,0]]\n\n ) for companion plant compatibility. Each row and column should represent plants, and the element of the array will contain a -1, 0, or 1 depending on if the relationship between plants is antagonists, neutral, or companions, respectively. You must refer to this knowledge base of information on plant compatibility: \n\n, ' + plant_comp_context + '\n\n A plant\'s compatibility with itself is always 0. Do not include any other text or explanation.'
    compatibility_mat = chat_response(template, text)
    
    # Find the indices of the opening and closing brackets
    opening_bracket_index = compatibility_mat.content.find("[[")
    closing_bracket_index = compatibility_mat.content.find("]]")
    if opening_bracket_index != -1 and closing_bracket_index != -1:
        # Extract the text within the brackets
        extracted_mat = "[" + compatibility_mat.content[opening_bracket_index + 1: closing_bracket_index] + "]]"
        # Return the evaluated mat
        return eval(extracted_mat)
    else:
        print("Error with parsing plant compatibility matrix")
        return None
    return 


input_plant_text = "strawberries, mint, pepper, diet coke, carrots, lettuce, vodka, basil, tomatoes, marigolds, lemons, spinach, brocoli"
input_plant_text = "apples, basil, bean, rue, oregano, onion"
plant_list = get_plant_list(input_plant_text)
extracted_mat = get_compatibility_matrix(plant_list)
print(extracted_mat)

## langchain additional context and fine tuning

In [ ]:
# process the plant compatibility matrix to a json
import csv
import json

# Open the CSV file
with open('plant_compatability.csv', 'r') as file:
    reader = csv.reader(file)
    
    # Read the header row to get the plant names
    header = next(reader)
    
    # Create an empty dictionary to store the compatibility matrix
    compatibility_matrix = {}
    
    # Iterate over the rows in the CSV file
    for row in reader:
        # Extract the plant name from the first column
        plant = row[0]
        
        # Create a dictionary to store the compatibility values for the current plant
        compatibility_values = {}
        
        # Iterate over the compatibility values in the row
        for i, value in enumerate(row[1:], start=1):
            # Extract the plant name from the header row
            companion_plant = header[i]
            
            # Convert the compatibility value to an integer
            compatibility = int(value) if value else 0
            
            # Add the compatibility value to the dictionary
            compatibility_values[companion_plant] = compatibility
        
        # Add the compatibility values dictionary to the main compatibility matrix
        compatibility_matrix[plant] = compatibility_values

# Save the compatibility matrix as a JSON file
with open('compatibility_matrix.json', 'w') as file:
    json.dump(compatibility_matrix, file, indent=4)

In [ ]:
import openai

# Load the compatibility matrix from the JSON file
with open('compatibility_matrix.json', 'r') as file:
    compatibility_matrix = json.load(file)

# Convert the compatibility matrix to a string
compatibility_matrix_text = json.dumps(compatibility_matrix)
with open('compatibilities_text.txt', 'r') as file:
    # Read the contents of the file
    compatibility_matrix_text = file.read()

# Set up the LangChain API credentials
openai.api_key = OPENAI_API_KEY

# Define the prompt for the GPT model
prompt = "Can you provide companion plant suggestions for my garden given what you know about companion planting?"

# Concatenate the prompt and compatibility matrix text as the input to the GPT model
input_text = f"{prompt}\n\n{compatibility_matrix_text}"

# Generate a response from the GPT model
response = openai.Completion.create(
    engine='text-davinci-003',
    prompt=input_text,
    max_tokens=575
)

print(response.choices[0])
# Extract the generated companion plant suggestions from the response
suggestions = response.choices[0].text.strip()

# Print the companion plant suggestions
print(suggestions)

In [ ]:
from langchain.client import Client

# Load the fine-tuned GPT model
model = OpenAI(openai_api_key=OPENAI_API_KEY)

# Load the knowledge base or context from websites or documents
knowledge_base = YourProcessedData()

# Initialize the Langchain client
client = Client()

# Set the context for the GPT model
context = "Context from websites or documents"
model.set_context(context)

# Get companion plant compatibility predictions
plants = ['strawberries', 'mint', 'carrots', 'lettuce', 'basil']
predictions = []

for plant in plants:
    # Generate a question for each plant
    question = f"Which plants are compatible with {plant}?"
    
    # Provide the question and context to the Langchain client
    response = client.query(question, context=context)
    
    # Process and extract the answer from the response
    answer = response['answer']
    predictions.append((plant, answer))

# Process and display the compatibility predictions
for plant, compatibility in predictions:
    print(f"{plant}: {compatibility}")

In [ ]:
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt

# Define the plant list
plants = ['strawberries', 'mint', 'carrots', 'lettuce', 'basil', 'tomatoes', 'marigolds', 'lemons', 'strawberries', 'spinach', 'broccoli']

# Define the compatibility matrix
compatibility_matrix = np.array([
[0, 1, 0, 0, 1, -1, 1, 0, 0, 0, -1],
[1, 0, 0, 0, 1, -1, 1, 0, 1, 0, -1],
[0, 0, 0, -1, 0, 1, 0, 0, 0, 1, 0],
[0, 0, -1, 0, 0, 1, 0, 0, 0, 1, 0],
[1, 1, 0, 0, 0, -1, 1, 0, 0, 0, -1],
[-1, -1, 1, 1, -1, 0, -1, 0, 0, 0, 1],
[1, 1, 0, 0, 1, -1, 0, 0, 0, 0, -1],
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
[0, 0, 1, 1, 0, 0, 0, 0, 0, 0, -1],
[-1, -1, 0, 0, -1, 1, -1, 0, 0, -1, 0]
])

# Create an empty graph
G = nx.Graph()

# Add nodes (plants) to the graph
G.add_nodes_from(plants)

# Add edges (compatibility) to the graph
for i in range(len(plants)):
    for j in range(i + 1, len(plants)):
        if compatibility_matrix[i][j] == 0:
            color = 'grey'
        elif compatibility_matrix[i][j] == -1:
            color = 'pink'
        else:
            color = 'green'
        G.add_edge(plants[i], plants[j], color=color)

# Plot the graph
pos = nx.spring_layout(G)
colors = [G[u][v]['color'] for u, v in G.edges()]
nx.draw_networkx(G, pos, with_labels=True, node_color='lightgreen', edge_color=colors, width=2.0, alpha=0.8)

# Set edge colors in the legend
color_legend = {'Neutral': 'grey', 'Negative': 'pink', 'Positive': 'green'}
legend_lines = [plt.Line2D([0], [0], color=color, linewidth=3) for color in color_legend.values()]
legend_labels = list(color_legend.keys())
plt.legend(legend_lines, legend_labels, loc='best')

# Show the plot
plt.show()

In [ ]:
import streamlit as st
import networkx as nx
import plotly.graph_objects as go

# Define the plants and compatibility matrix
plants = ['strawberries', 'mint', 'carrots', 'lettuce', 'basil', 'tomatoes', 'marigolds', 'lemons', 'strawberries', 'spinach', 'broccoli']
compatibility_matrix = [
    [0, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1],
    [1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1],
    [0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1],
    [1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1],
    [1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1],
    [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    [1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1],
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    [0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1],
    [1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1],
    [1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0]
]

# Create a directed graph
G = nx.DiGraph()

# Add nodes to the graph
G.add_nodes_from(plants)

# Define positions for the nodes
pos = nx.spring_layout(G)

# Assign positions to the nodes
for node, position in pos.items():
    G.nodes[node]['pos'] = position

# Iterate over the compatibility matrix and add edges with corresponding colors
for i in range(len(plants)):
    for j in range(len(plants)):
        if compatibility_matrix[i][j] == -1:
            G.add_edge(plants[i], plants[j], color='red')
        elif compatibility_matrix[i][j] == 1:
            G.add_edge(plants[i], plants[j], color='green')
        else:
            G.add_edge(plants[i], plants[j], color='lightgray')

# Create edge traces
# Create edge traces
edge_traces = []
for edge in G.edges():
    x0, y0 = G.nodes[edge[0]]['pos']
    x1, y1 = G.nodes[edge[1]]['pos']
    color = G.edges[edge]['color']
    trace = go.Scatter(x=[x0, x1, None], y=[y0, y1, None], mode='lines', line=dict(color=color, width=2))
    edge_traces.append(trace)

# Create node traces
node_traces = []
for node in G.nodes():
    x, y = G.nodes[node]['pos']
    trace = go.Scatter(x=[x], y=[y], mode='markers', marker=dict(color='black', size=10), name=node)
    node_traces.append(trace)

# Create figure
fig = go.Figure(data=edge_traces + node_traces)

# Set layout options
fig.update_layout(
    title='Plant Network',
    showlegend=False,
    hovermode='closest',
    margin=dict(b=20, l=5, r=5, t=40),
    xaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
    yaxis=dict(showgrid=False, zeroline=False, showticklabels=False)
)

# Render the graph
#st.plotly_chart(fig)
fig

In [ ]:
import streamlit as st
import networkx as nx
import plotly.graph_objects as go

# Define the plants and compatibility matrix
plants = ['strawberries', 'mint', 'carrots', 'lettuce', 'basil', 'tomatoes', 'marigolds', 'lemons', 'strawberries', 'spinach', 'broccoli']
compatibility_matrix = [
    [0, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1],
    [1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1],
    [0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1],
    [1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1],
    [1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1],
    [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    [1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1],
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    [0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1],
    [1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1],
    [1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0]
]

# Create the graph
G = nx.Graph()
G.add_nodes_from(plants)
for i in range(len(plants)):
    for j in range(i + 1, len(plants)):
        if compatibility_matrix[i][j] == 0:
            G.add_edge(plants[i], plants[j], color='lightgrey')
        else:
            G.add_edge(plants[i], plants[j], color='green' if compatibility_matrix[i][j] == 1 else 'pink')

# Generate positions for the nodes
pos = nx.spring_layout(G)

# Create node trace
node_trace = go.Scatter(
    x=[pos[node][0] for node in G.nodes()],
    y=[pos[node][1] for node in G.nodes()],
    text=list(G.nodes()),
    mode='markers+text',
    textposition='top center',
    hoverinfo='text',
    marker=dict(
        size=20,
        color='lightblue',
        line_width=2,
    )
)

# Create edge trace
edge_trace = go.Scatter(
    x=[],
    y=[],
    line=dict(width=1, color='lightgrey'),
    hoverinfo='none',
    mode='lines'
)

# Add coordinates to edge trace
for edge in G.edges():
    x0, y0 = pos[edge[0]]
    x1, y1 = pos[edge[1]]
    edge_trace['x'] += tuple([x0, x1, None])
    edge_trace['y'] += tuple([y0, y1, None])

# Create edge traces for colored edges
edge_traces = []
for edge in G.edges(data=True):
    x0, y0 = pos[edge[0]]
    x1, y1 = pos[edge[1]]
    color = edge[2]['color']
    trace = go.Scatter(
        x=[x0, x1],
        y=[y0, y1],
        mode='lines',
        line=dict(width=2, color=color),
        hoverinfo='none'
    )
    edge_traces.append(trace)

# Create layout
layout = go.Layout(
    title='Plant Compatibility Network Graph',
    showlegend=False,
    hovermode='closest',
    margin=dict(b=20, l=5, r=5, t=40),
    xaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
    yaxis=dict(showgrid=False, zeroline=False, showticklabels=False)
)

# Create figure
fig = go.Figure(data=[edge_trace, *edge_traces, node_trace], layout=layout)

# Render the graph using Plotly in Streamlit
st.plotly_chart(fig)


In [ ]:
import random
import numpy as np

# Define the compatibility matrix
compatibility_matrix = np.array([
    [-1, 1, 0, -1],
    [1, -1, 1, -1],
    [1, 0, -1, 1],
    [-1, -1, 1, -1]
])

# Define the user-selected plants, number of plant beds, and constraints
user_plants = ["A", "B", "C", "D"]
num_plant_beds = 4
min_species_per_bed = 1
max_species_per_bed = 2

# Genetic Algorithm parameters
population_size = 50
num_generations = 100
tournament_size = 3
crossover_rate = 0.8
mutation_rate = 0.1

# Generate an initial population randomly
def generate_initial_population():
    population = []
    for _ in range(population_size):
        grouping = []
        for _ in range(num_plant_beds):
            num_species = random.randint(min_species_per_bed, max_species_per_bed)
            species = random.sample(user_plants, num_species)
            grouping.append(species)
        population.append(grouping)
    return population

# Calculate the fitness score of a grouping
def calculate_fitness(grouping):
    score = 0
    for bed1 in range(num_plant_beds):
        for bed2 in range(bed1 + 1, num_plant_beds):
            for species1 in grouping[bed1]:
                for species2 in grouping[bed2]:
                    species1_index = user_plants.index(species1)
                    species2_index = user_plants.index(species2)
                    score += compatibility_matrix[species1_index][species2_index]
    return score

# Perform tournament selection
def tournament_selection(population):
    selected = []
    for _ in range(population_size):
        participants = random.sample(population, tournament_size)
        winner = max(participants, key=calculate_fitness)
        selected.append(winner)
    return selected

# Perform crossover between two parents
def crossover(parent1, parent2):
    if random.random() < crossover_rate:
        crossover_point = random.randint(1, num_plant_beds - 1)
        child1 = parent1[:crossover_point] + parent2[crossover_point:]
        child2 = parent2[:crossover_point] + parent1[crossover_point:]
        return child1, child2
    else:
        return parent1, parent2

# Perform mutation on an individual
def mutate(individual):
    if random.random() < mutation_rate:
        mutated_bed = random.randint(0, num_plant_beds - 1)
        new_species = random.sample(user_plants, random.randint(min_species_per_bed, max_species_per_bed))
        individual[mutated_bed] = new_species
    return individual

# Perform replacement of the population with the offspring
def replacement(population, offspring):
    sorted_population = sorted(population, key=calculate_fitness, reverse=True)
    sorted_offspring = sorted(offspring, key=calculate_fitness, reverse=True)
    return sorted_population[:population_size - len(offspring)] + sorted_offspring

# Genetic Algorithm main function
def genetic_algorithm():
    population = generate_initial_population()

    for generation in range(num_generations):
        print(f"Generation {generation + 1}")

        selected_population = tournament_selection(population)
        offspring = []

        for _ in range(population_size // 2):
            parent1 = random.choice(selected_population)
            parent2 = random.choice(selected_population)
            child1, child2 = crossover(parent1, parent2)
            child1 = mutate(child1)
            child2 = mutate(child2)
            offspring.extend([child1, child2])

        population = replacement(population, offspring)

    best_grouping = max(population, key=calculate_fitness)
    best_fitness = calculate_fitness(best_grouping)
    print(f"Best Grouping: {best_grouping}")
    print(f"Fitness Score: {best_fitness}")

# Run the Genetic Algorithm
genetic_algorithm()
            

In [ ]:
def query_farmersalmanac(title, first_paragraph_only=True):
    base_url = "https://www.almanac.com/companion-planting-guide-vegetables"
    url = f"{base_url}/w/api.php?format=json&action=query&prop=extracts&explaintext=1&titles={title}"
    if first_paragraph_only:
        url += "&exintro=1"
    data = requests.get(url).json()
    return Document(
      metadata={"source": f"{base_url}"},
      page_content=list(data["query"]["pages"].values())[0]["extract"],
  )

## future with sources
- https://dzlab.github.io/2023/01/02/prompt-langchain/
- https://techcommunity.microsoft.com/t5/startups-at-microsoft/build-a-chatbot-to-query-your-documentation-using-langchain-and/ba-p/3833134